In [91]:
import numpy as np
import matplotlib.pyplot as plt
import time
import random

In [92]:
# Partie 1: Implémentation des méthodes

def construire_matrice_td(termes, documents):
    """
    Construit une matrice termes-documents D où D[i,j] = 1 si le terme i est dans le document j
    """
    D = np.zeros((len(termes), len(documents)), dtype=int)
    for j, doc in enumerate(documents):
        for i, terme in enumerate(termes):
            if terme in doc:
                D[i, j] = 1
    return D

In [93]:
def construire_vecteur_requete(termes, requete):
    """
    Construit un vecteur de requête q où q[i] = 1 si le terme i est dans la requête
    """
    q = np.zeros(len(termes), dtype=int)
    for i, terme in enumerate(termes):
        if terme in requete:
            q[i] = 1
    return q

In [94]:
def calculer_scores_standard(D, q):
    """
    Calcule les scores de pertinence sans SVD
    s_j(q) = <q, d_j> / (||q||_2 * ||d_j||_2)
    """
    norme_q = np.linalg.norm(q)
    if norme_q == 0:
        return np.zeros(D.shape[1])
    
    scores = []
    for j in range(D.shape[1]):
        d_j = D[:, j]
        norme_d_j = np.linalg.norm(d_j)
        if norme_d_j == 0:
            scores.append(0)
        else:
            produit_scalaire = np.dot(q, d_j)
            score = produit_scalaire / (norme_q * norme_d_j)
            scores.append(score)
    return np.array(scores)

In [95]:
def decomposition_SVD(D):
    """
    Calcule directement la décomposition SVD d'une matrice D
    Retourne U, S, V^T tels que D = U * S * V^T
    """
    return np.linalg.svd(D, full_matrices=False)

In [96]:
def approximation_rang_k(U, S, Vt, k):
    """
    Calcule l'approximation de rang k d'une matrice à partir de sa SVD
    """
    k = min(k, U.shape[1], S.shape[0], Vt.shape[0])
    U_k = U[:, :k]
    S_k = S[:k, :k] if S.ndim == 2 else np.diag(S[:k])
    Vt_k = Vt[:k, :]
    return U_k, S_k, Vt_k

In [97]:
def calculer_scores_svd(U_k, S_k, Vt_k, q):
    """
    Calcule les scores de pertinence avec SVD
    s_j(q) = <U_k^T q, S_k V_k^T e_j> / (||U_k^T q||_2 * ||S_k V_k^T e_j||_2)
    """
    q_proj = np.dot(U_k.T, q)
    norme_q_proj = np.linalg.norm(q_proj)
    if norme_q_proj == 0:
        return np.zeros(Vt_k.shape[1])
    
    scores = []
    for j in range(Vt_k.shape[1]):
        e_j = np.zeros(Vt_k.shape[1])
        e_j[j] = 1
        d_j_proj = np.dot(S_k, np.dot(Vt_k, e_j))
        norme_d_j_proj = np.linalg.norm(d_j_proj)
        if norme_d_j_proj == 0:
            scores.append(0)
        else:
            produit_scalaire = np.dot(q_proj, d_j_proj)
            score = produit_scalaire / (norme_q_proj * norme_d_j_proj)
            scores.append(score)
    return np.array(scores)

In [98]:
# Partie 2: Bidiagonalisation et méthode QR
def bidiagonalisation(D):
    """
    Implémentation de l'algorithme de bidiagonalisation
    Retourne U, B, V tels que B = U^T * D * V est bidiagonale
    """
    n, m = D.shape
    r = min(n, m)
    
    U = np.zeros((n, r))
    V = np.zeros((m, r))
    B = np.zeros((r, r))
    
    v1 = np.random.rand(m)
    v1 = v1 / np.linalg.norm(v1)
    V[:, 0] = v1
    
    u_tilde = np.dot(D, v1)
    sigma1 = np.linalg.norm(u_tilde)
    if sigma1 < 1e-10:
        sigma1 = 1e-10
    u1 = u_tilde / sigma1
    U[:, 0] = u1
    B[0, 0] = sigma1
    
    for i in range(1, r):
        v_tilde = np.dot(D.T, U[:, i-1]) - B[i-1, i-1] * V[:, i-1]
        for j in range(i):
            v_tilde -= np.dot(v_tilde, V[:, j]) * V[:, j]
        beta_i = np.linalg.norm(v_tilde)
        if beta_i < 1e-10:
            beta_i = 1e-10
        v_i = v_tilde / beta_i
        V[:, i] = v_i
        B[i, i-1] = beta_i
        
        u_tilde = np.dot(D, v_i) - beta_i * U[:, i-1]
        for j in range(i):
            u_tilde -= np.dot(u_tilde, U[:, j]) * U[:, j]
        sigma_i = np.linalg.norm(u_tilde)
        if sigma_i < 1e-10:
            sigma_i = 1e-10
        u_i = u_tilde / sigma_i
        U[:, i] = u_i
        B[i, i] = sigma_i
    
    return U, B, V

In [99]:
def qr_factorisation(A):
    """
    Calcule la factorisation QR d'une matrice A
    """
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for j in range(n):
        v = A[:, j].copy()
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], A[:, j])
            v -= R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(v)
        if R[j, j] > 1e-10:
            Q[:, j] = v / R[j, j]
        else:
            Q[:, j] = np.zeros(m)
    return Q, R

In [100]:
def methode_qr_bidiagonale(B, max_iterations=100, tol=1e-8):
    """
    Méthode QR pour extraire les valeurs singulières d'une matrice bidiagonale
    """
    B_k = B.copy()
    n = B.shape[0]
    
    for k in range(max_iterations):
        if np.max(np.abs(np.tril(B_k, -1))) < tol:
            break
        Q_k, R_k = qr_factorisation(B_k.T)
        Q_tilde_k, B_k_plus_1 = qr_factorisation(R_k.T)
        B_k = B_k_plus_1.T
    
    valeurs_singulieres = np.abs(np.diag(B_k))
    indices = np.argsort(valeurs_singulieres)[::-1]
    valeurs_singulieres = valeurs_singulieres[indices]
    return valeurs_singulieres

In [101]:
def decomposition_SVD_bidiag_QR(D):
    """
    Calcule la décomposition SVD en utilisant la bidiagonalisation suivie de QR
    """
    U, B, V = bidiagonalisation(D)
    valeurs_singulieres = methode_qr_bidiagonale(B)
    
    r = min(U.shape[1], V.shape[1], len(valeurs_singulieres))
    S = np.zeros((r, r))
    np.fill_diagonal(S, valeurs_singulieres[:r])
    
    return U, S, V.T

In [102]:
def calculer_scores_bidiag_qr(U_k, S_k, Vt_k, q):
    """
    Calcule les scores de pertinence pour la méthode bidiag+QR
    """
    return calculer_scores_svd(U_k, S_k, Vt_k, q)

In [103]:
def erreur_reconstruction(D, D_k):
    """
    Calcule l'erreur de reconstruction ||D - D_k||_2
    """
    return np.linalg.norm(D - D_k, ord=2)

In [104]:
def afficher_classement(scores, seuil=0, prefix="D"):
    """
    Affiche le classement des documents par score de pertinence
    """
    classement = sorted(enumerate(scores, start=1), key=lambda x: x[1], reverse=True)
    print(f"\nClassement des documents (score > {seuil}) :")
    for doc_id, score in classement:
        if score > seuil:
            print(f"{prefix}{doc_id} : {score:.2f}")

In [105]:

# Partie 3: Tests et analyses
def exemple1():
    #Test pour l'exemple 1 avec la requête q1 = {matrices, algèbre}

    print("\n==== EXEMPLE 1 ====")
    
    termes = ["algebre", "analyse", "determinants", "matrices", "probabilites", "statistiques", "suites"]
    documents = [
        ["algebre", "lineaire", "matrices"],
        ["analyse", "reelle", "suites"],
        ["probabilites", "statistiques"],
        ["matrices", "determinants"]
    ]
    
    D = construire_matrice_td(termes, documents)
    print("Matrice D (termes-documents) :")
    print(D)
    
    requete = ["matrices", "algebre"]
    q = construire_vecteur_requete(termes, requete)
    print("\nVecteur de requête q1 :", q)
    
    # Méthode standard
    print("\n--- Méthode standard (sans SVD) ---")
    scores_standard = calculer_scores_standard(D, q)
    afficher_classement(scores_standard)
    
    # Méthode SVD directe
    print("\n--- Méthode SVD directe ---")
    for k in range(1, min(D.shape[0], D.shape[1]) + 1):
        U, S, Vt = decomposition_SVD(D)
        U_k, S_k, Vt_k = approximation_rang_k(U, S, Vt, k)
        scores_svd = calculer_scores_svd(U_k, S_k, Vt_k, q)
        D_k = np.dot(U_k, np.dot(S_k, Vt_k))
        err = erreur_reconstruction(D, D_k)
        print(f"\nk = {k}, Erreur de reconstruction: {err:.6f}")
        afficher_classement(scores_svd)
    
    # Méthode Bidiagonalisation + QR
    print("\n--- Méthode Bidiagonalisation + QR ---")
    for k in range(1, min(D.shape[0], D.shape[1]) + 1):
        U, S, Vt = decomposition_SVD_bidiag_QR(D)
        U_k, S_k, Vt_k = approximation_rang_k(U, S, Vt, k)
        scores_bidiag_qr = calculer_scores_bidiag_qr(U_k, S_k, Vt_k, q)
        D_k = np.dot(U_k, np.dot(S_k, Vt_k))
        err = erreur_reconstruction(D, D_k)
        print(f"\nk = {k}, Erreur de reconstruction: {err:.6f}")
        afficher_classement(scores_bidiag_qr)


In [106]:
def exemple3():
    """
    Test pour l'exemple 3 avec les requêtes:
    - q2 = {dépression, croissance}
    - q3 = {bassin, fiscalité}
    """
    print("\n==== EXEMPLE 3 ====")
    
    termes = ["bassin", "chomage", "commerce", "couches", "croissance", "depression",
              "emploi", "erosion", "exportation", "faille", "fiscalite", "forage",
              "geologie", "gisement", "impots", "inflation", "investissement",
              "monnaie", "pib", "plaque", "revenu", "roche", "salaires", "seisme",
              "stratigraphie", "tectonique", "tremblement", "volcan"]
    
    documents = [
        ["croissance", "pib", "investissement"],
        ["inflation", "monnaie", "depression"],
        ["commerce", "exportation", "croissance"],
        ["emploi", "chomage", "salaires"],
        ["impots", "fiscalite", "revenu"],
        ["geologie", "faille", "tremblement"],
        ["volcan", "seisme", "plaque", "tectonique"],
        ["depression", "bassin", "erosion"],
        ["stratigraphie", "couches", "roche"],
        ["gisement", "forage", "bassin"]
    ]
    
    D = construire_matrice_td(termes, documents)
    print("Matrice termes-documents pour l'exemple 3 (dimensions):", D.shape)
    
    requetes = [
        ["depression", "croissance"],
        ["bassin", "fiscalite"]
    ]
    
    for i, requete in enumerate(requetes, 2):
        print(f"\n==== Requête q{i} = {requete} ====")
        q = construire_vecteur_requete(termes, requete)
        
        # Méthode standard
        print("\n--- Méthode standard (sans SVD) ---")
        scores_standard = calculer_scores_standard(D, q)
        afficher_classement(scores_standard, seuil=0.5)
        
        # Méthode SVD directe
        print("\n--- Méthode SVD directe ---")
        erreurs = []
        for k in range(1, min(D.shape) + 1):
            U, S, Vt = decomposition_SVD(D)
            U_k, S_k, Vt_k = approximation_rang_k(U, S, Vt, k)
            D_k = np.dot(U_k, np.dot(S_k, Vt_k))
            err = erreur_reconstruction(D, D_k)
            erreurs.append(err)
            if k == 3:
                scores_svd = calculer_scores_svd(U_k, S_k, Vt_k, q)
                print(f"\nk = {k}, Erreur de reconstruction: {err:.6f}")
                afficher_classement(scores_svd, seuil=0.5)
        
        # Méthode Bidiagonalisation + QR
        print("\n--- Méthode Bidiagonalisation + QR ---")
        U, S, Vt = decomposition_SVD_bidiag_QR(D)
        for k in [3]:
            U_k, S_k, Vt_k = approximation_rang_k(U, S, Vt, k)
            scores_bidiag_qr = calculer_scores_bidiag_qr(U_k, S_k, Vt_k, q)
            D_k = np.dot(U_k, np.dot(S_k, Vt_k))
            err = erreur_reconstruction(D, D_k)
            print(f"\nk = {k}, Erreur de reconstruction: {err:.6f}")
            afficher_classement(scores_bidiag_qr, seuil=0.5)
        
        # Plot reconstruction error
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(erreurs) + 1), erreurs, 'b-o')
        plt.title(f"Erreur de reconstruction ||D - D_k||_2 pour la requête q{i}")
        plt.xlabel("Rang k")
        plt.ylabel("Erreur")
        plt.grid(True)
        plt.savefig(f'reconstruction_error_q{i}.png')
        plt.close()


In [107]:
def analyser_performances():
    """
    Analyse des performances en temps d'exécution en fonction de la taille des matrices
    """
    print("\n==== ANALYSE DES PERFORMANCES ====")
    
    Nd_values = [5, 10, 15, 20, 25, 30]
    Nt_values = [3 * Nd for Nd in Nd_values]
    
    temps_standard = []
    temps_svd = []
    temps_bidiag_qr = []
    
    for Nd, Nt in zip(Nd_values, Nt_values):
        print(f"\nTest avec matrice de taille {Nt}x{Nd}")
        D = np.random.randint(0, 2, size=(Nt, Nd))
        q = np.random.randint(0, 2, size=Nt)
        
        # Méthode standard
        start_time = time.time()
        _ = calculer_scores_standard(D, q)
        temps_standard.append(time.time() - start_time)
        
        # Méthode SVD directe
        start_time = time.time()
        U, S, Vt = decomposition_SVD(D)
        U_k, S_k, Vt_k = approximation_rang_k(U, S, Vt, min(5, min(Nt, Nd)))
        _ = calculer_scores_svd(U_k, S_k, Vt_k, q)
        temps_svd.append(time.time() - start_time)
        
        # Méthode Bidiagonalisation + QR
        start_time = time.time()
        U, S, Vt = decomposition_SVD_bidiag_QR(D)
        U_k, S_k, Vt_k = approximation_rang_k(U, S, Vt, min(5, min(Nt, Nd)))
        _ = calculer_scores_bidiag_qr(U_k, S_k, Vt_k, q)
        temps_bidiag_qr.append(time.time() - start_time)
    
    plt.figure(figsize=(12, 8))
    plt.plot(Nd_values, temps_standard, 'b-o', label='Méthode standard')
    plt.plot(Nd_values, temps_svd, 'r-s', label='SVD directe')
    plt.plot(Nd_values, temps_bidiag_qr, 'g-^', label='Bidiagonalisation + QR')
    plt.title("Temps d'exécution en fonction de la taille de la matrice")
    plt.xlabel("Nombre de documents (Nd)")
    plt.ylabel("Temps (secondes)")
    plt.legend()
    plt.grid(True)
    plt.savefig('performance_analysis.png')
    plt.close()

In [108]:
def traiter_fichier_documents(chemin_fichier):
    """
    Traite le fichier documents.txt pour extraire la matrice termes-documents
    """
    try:
        with open(chemin_fichier, 'r', encoding='utf-8') as file:
            contenu = file.read()
        
        documents_bruts = [line.strip() for line in contenu.split('\n') if line.strip()]
        termes_set = set()
        documents = []
        
        for doc in documents_bruts:
            mots = [mot.lower().strip('.,;:!?()[]{}\"\'') for mot in doc.split()]
            mots = [mot for mot in mots if mot]
            termes_set.update(mots)
            documents.append(mots)
        
        termes = sorted(list(termes_set))
        print(f"Nombre de documents: {len(documents)}")
        print(f"Nombre de termes: {len(termes)}")
        
        D = construire_matrice_td(termes, documents)
        return termes, documents, D
    
    except FileNotFoundError:
        print(f"Erreur: Le fichier {chemin_fichier} n'a pas été trouvé.")
        return [], [], np.array([])

In [109]:
def test_fichier_documents(chemin_fichier):
    """
    Test avec le fichier documents.txt
    """
    print("\n==== TEST AVEC LE FICHIER DOCUMENTS.TXT ====")
    
    termes, documents, D = traiter_fichier_documents(chemin_fichier)
    
    if len(termes) == 0:
        print("Utilisation d'un exemple simplifié...")
        termes = ["algebre", "analyse", "matrices", "statistiques"]
        documents = [
            ["algebre", "matrices"],
            ["analyse", "suites"],
            ["statistiques"],
            ["matrices"]
        ]
        D = construire_matrice_td(termes, documents)
    
    mots_requete = random.sample(termes, min(3, len(termes)))
    print(f"\nRequête aléatoire: {mots_requete}")
    
    q = construire_vecteur_requete(termes, mots_requete)
    
    # Méthode standard
    print("\n--- Méthode standard (sans SVD) ---")
    scores_standard = calculer_scores_standard(D, q)
    top_docs = np.argsort(scores_standard)[::-1][:5]
    print("\nTop 5 documents:")
    for i, doc_id in enumerate(top_docs, 1):
        print(f"{i}. Document {doc_id+1}: Score = {scores_standard[doc_id]:.4f}")
        print(f"   Contenu: {' '.join(documents[doc_id][:10])}...")
    
    # Méthode SVD directe
    print("\n--- Méthode SVD directe (k=5) ---")
    k = min(5, min(D.shape))
    U, S, Vt = decomposition_SVD(D)
    U_k, S_k, Vt_k = approximation_rang_k(U, S, Vt, k)
    scores_svd = calculer_scores_svd(U_k, S_k, Vt_k, q)
    top_docs_svd = np.argsort(scores_svd)[::-1][:5]
    
    print("\nTop 5 documents (SVD):")
    for i, doc_id in enumerate(top_docs_svd, 1):
        print(f"{i}. Document {doc_id+1}: Score = {scores_svd[doc_id]:.4f}")
        print(f"   Contenu: {' '.join(documents[doc_id][:10])}...")
    
    print("\n--- Comparaison des méthodes ---")
    print(f"Documents communs entre les deux méthodes: {len(set(top_docs) & set(top_docs_svd))}")

In [110]:
# Programme principal
if __name__ == "__main__":
    np.random.seed(42)  # For reproducibility
    exemple1()
    exemple3()
    analyser_performances()
    test_fichier_documents("documents.txt")
    print("\n==== TESTS TERMINÉS ====")


==== EXEMPLE 1 ====
Matrice D (termes-documents) :
[[1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [1 0 0 1]
 [0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]]

Vecteur de requête q1 : [1 0 0 1 0 0 0]

--- Méthode standard (sans SVD) ---

Classement des documents (score > 0) :
D1 : 1.00
D4 : 0.50

--- Méthode SVD directe ---

k = 1, Erreur de reconstruction: 1.414214

Classement des documents (score > 0) :
D1 : 1.00
D4 : 1.00

k = 2, Erreur de reconstruction: 1.414214

Classement des documents (score > 0) :
D1 : 1.00
D4 : 1.00

k = 3, Erreur de reconstruction: 1.000000

Classement des documents (score > 0) :
D1 : 1.00
D4 : 1.00

k = 4, Erreur de reconstruction: 0.000000

Classement des documents (score > 0) :
D1 : 1.00
D4 : 0.50

--- Méthode Bidiagonalisation + QR ---

k = 1, Erreur de reconstruction: 1.660011

Classement des documents (score > 0) :
D1 : 1.00
D2 : 1.00
D3 : 1.00
D4 : 1.00

k = 2, Erreur de reconstruction: 1.414214

Classement des documents (score > 0) :
D1 : 0.90
D4 : 0.49
D2 : 0.36
D3 : 0.36

k = 